In [ ]:
import arcticdb as adb
import json
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import dtale
from sklearn.decomposition import PCA
from tradingo import backtest
from tradingo.utils import get_instruments, get_config, with_instrument_details
from tradingo.api import Tradingo
import itertools
import seaborn as sns
from tradingo.plotting import plot_multi
import logging
from tradingo import sampling

In [ ]:
t = Tradingo(name="etft", uri="lmdb:///home/rory/dev/airflow/test/arctic.db", provider="yfinance", universe='vanguard')

In [ ]:
t.

In [ ]:
import logging
logging.getLogger("tradingo").setLevel(logging.INFO)

In [ ]:
t.backtest.trend.raw.shares

In [ ]:

t.backtest.trend

In [ ]:
sampling.sample_equity(
    start_date=pd.Timestamp("2024-01-01"),
    end_date=pd.Timestamp.now().normalize(),
    name='etfs',
    config_name='etft',
    provider='yfinance',
    universe='etfs',
)

In [ ]:
bt = backtest.backtest(start_date=pd.Timestamp("2021-01-01 00:00:00+00:00"), end_date=pd.Timestamp.now("utc"), name="trend", stage="raw", config_name="ETFT", provider="yfinance", dry_run=True)

In [ ]:
(np.sqrt(260) * ((1+t.prices.close().pct_change()).prod(axis=1).pow(1/398) - 1).ewm(10).std()).plot()

In [ ]:
ivol = t.prices.ivol()

In [ ]:
top_ivol = ivol.iloc[-1].rename("ivol").sort_values(ascending=False).to_frame().merge(t.instruments.etfs(), left_index=True, right_index=True)[["ivol", "Name"]].head(40)

In [ ]:
pxs=t.prices.adj_close()
# pxs.plot()
pxs[['IBLC', 'EWZS', 'ITB', 'ETEC', 'BTEK', 'ILIT', 'IRBO']].plot()
(top_ivol.ivol * 100)


In [ ]:
vol = lambda uni: (1 - ((1 + uni).prod(axis=1).pow(1 / 100))).ewm(10).std()
vol(prices.pct_change())

In [ ]:
("ETFT", "trend", "portfolio", "summary")

In [ ]:
from tradingo.utils import with_instrument_details
RETURNS = with_instrument_details(instruments=t.instruments.etfs(),
                        dataframe=t.prices.adj_close().pct_change(),
                        columns=["Asset Class"])

In [ ]:
(1+RETURNS.loc[:,"Commodities & Digital Assets"].transpose().merge(t.instruments.etfs().Name, left_index=True, right_index=True).reset_index(drop=True).set_index(["Name"]).transpose()).cumprod().plot(figsize=(12, 6))

In [ ]:
ivol = with_instrument_details(instruments=t.instruments.etfs(), dataframe=t.prices.ivol(), columns=["Asset Class"]).groupby("Asset Class", axis=1).idxmax(axis=1)
ivol_selection = ivol.iloc[-100]

In [ ]:
close = with_instrument_details(instruments=t.instruments.etfs(), dataframe=t.prices.close(), columns=["Asset Class"])

In [ ]:
close[ivol_selection.values].plot()

In [ ]:
pd.concat([pd.DataFrame([i.to_dict() for i in j]) for j in bt[-1]], axis=1, keys=trades.columns)

In [ ]:
df = pd.concat((
    t.prices.adj_close(), t.signals.trend.ewmac_64_32(), t.signals.trend.ewmac_128_64(), t.portfolio.trend.raw.shares()
),
               axis=1, keys=("PRICE", "SIGNAL", "ewmac_46_32", "ewmac_128_64", "SHARES")).loc[(slice(None), slice(None))]

In [ ]:
plot_multi(df.loc[:,(slice(None), "AAXJ")])

In [ ]:
corr = t.prices.close().corr().unstack()
corr[
    corr.index.get_level_values(0) != corr.index.get_level_values(1)
]
pairs = [i for i in itertools.combinations(corr.index.get_level_values(0).drop_duplicates(), 2)]
corr = corr.reindex(pd.MultiIndex.from_tuples(pairs)).idxmax()

In [ ]:
corr.idxmax()

In [ ]:
t.prices.

In [ ]:
t.instruments.etfs.head().loc["ITOT"]

In [ ]:
t.prices.close()

In [ ]:
t.prices.close()[t.prices.ivol().iloc[-1].idxmax()].plot(label=t.prices.ivol[-1].idxmax())

In [ ]:
t.instruments.etfs()

In [ ]:
(t.backtest.trend.portfolio().total_pnl+ 70_000).plot()

In [ ]:
position * price

In [ ]:

from tradingo.test import test_backtest as test
price = test.prices.__wrapped__()
position = test.position.__wrapped__(price)

bt = backtest.backtest(
    portfolio=position,                   
    prices=price,                
    name="test",                                   
    config_name="test",                            
    dry_run=True,                                  
     start_date=price.index[0],   
     end_date=price.index[-1],    
     stage="raw",                                   
   )

In [ ]:
bt['backtest/portfolio']

In [ ]:
t.instruments.etfs()[["Net Assets (USD)", "Name", "Region"]]

In [ ]:
def sharpe(returns: pd.DataFrame):
    return (1+returns).resample("M").prod()/returns.resample("M").std() * np.sqrt(260)

(((t.prices.close().pct_change()).resample("Y").mean())/(t.prices.close().pct_change().resample("Y").std())).IVV

In [ ]:
half_life = 62

mean_return = t.prices.close().pct_change().ewm(half_life).mean()
std_return = t.prices.close().pct_change().ewm(half_life).std()
(mean_return/std_return).IVV.iloc[62:].plot()


In [ ]:
(returns.mean()/returns.std())

In [ ]:
t.instruments.etfs().Name.loc["GOVZ"]

In [ ]:
t.prices.adj_close().transpose().sort_values("2024-05-24").tail(20).transpose().rename(instruments.Name.loc.__getitem__, axis=1).plot()

In [ ]:
corr_ivv = t.prices.adj_close().corr()["IVV"].sort_values()

In [ ]:
t.instruments.etfs().merge(corr_ivv.rename("corr_sp500"), left_index=True, right_index=True).sort_values("corr_sp500").head(10).Name

In [ ]:
t.signals.trend.vol_64()["UAE"]

In [ ]:
t.prices.adj_close()[["IVV", "IVW", ]].plot()

In [ ]:
instruments[
    ~instruments.Name.str.contains("Treasury")
]

In [ ]:
t.prices.

In [ ]:
pd.concat(
    (
        corr_ivv, t.prices.close().pct_change().mean().rename('median'),
        t.prices.close().pct_change().mean().rename('mean'),
        instruments.Name
    ),
    axis=1
).query("IVV > 0").plot.scatter(
    x='IVV',
    y='mean'
)

In [ ]:
instruments = t.instruments.etfs()
instruments[instruments.index.isin(total_return.tail(10).index)].transpose()

In [ ]:
ivol = t.prices.ivol().rename_axis("Symbol")
inst = t.instruments.etfs()
ivol.transpose().sort_values(ivol.index[-1]).merge(inst["Asset Class"]).set_index(["Asset Class"], append=True).reoorder_levels()

In [ ]:
t.instruments.etfs()

In [ ]:
with t.common_args(columns=["ACWV", "IVV", "IWFH"]):
    acwv = t.portfolio.trend.raw.shares()
bounds = pd.concat(
    (
        acwv,
        acwv.mul(1.1).shift(),
        acwv.mul(0.9).shift(),
    ),
    axis=1,
    keys=("unbuffered", "upper", "lower")
)
bounds = bounds.merge(pd.concat((bounds.unbuffered.where((
    ~((bounds.lower < bounds.unbuffered) & (bounds.unbuffered < bounds.upper))
), np.nan).ffill(),), keys=("buffered",), axis=1), right_index=True, left_index=True)
bounds.plot(figsize=(12,6))

In [ ]:
t.instruments.etfs()

In [ ]:
bounds.loc[:,(slice(None),"IVV")].plot(figsize=(12,6))

In [ ]:
inst_pnl = t.backtest.trend.raw.shares.instrument.total_pnl()
backtest = t.backtest.trend.raw.shares.buffered.instrument

instruments = t.instruments.etfs()
inst_pnl.transpose().sort_values(by=inst_pnl.index[-1], ascending=False).merge(instruments.Name, left_index=True, right_index=True).set_index("Name", append=True).transpose()[["IYW","IGV"]].plot()

In [ ]:
backtest

In [ ]:
data = pd.concat((t.backtest.trend.raw.shares.instrument.total_pnl(), (t.backtest.trend.raw.shares.instrument.net_position())), axis=1, keys=("total_pnl", "net_position"))

In [ ]:
from tradingo.plotting import plot_multi
plot_multi(data.loc[:, (slice(None),"IVV")])

In [ ]:
t.backtest.trend.raw.shares.portfolio().total_pnl.plot()